## Data 
First attempt to write a class for the data

In [ ]:
data = [] # save the data in this list 
# loop over files, each file is a 2 min walk for a single individual
for i, name in tqdm(enumerate(os.listdir(csvD_path))):

  id = name.split('_')[0]
  
  level = dem_df['HoehnYahr'][dem_df['ID'] == id ].values[0]
  
  if 'Co' in name:  
    parkinson = 0
  elif 'Pt' in name:
    parkinson = 1
  
  # reading each file, skipping 3 columns (time,total right forces & total left forces) to be calculated after filtering the data.
  # For gait cycle segmentation, to eliminate the effect of gait initiation and termenation, the first and last 20 seconds of VGRF data was discarded.
  one_walk = pd.read_csv(csvD_path+'/' + name, skiprows=list(range(1,21)) , skipfooter=20, usecols = [i for i in range(1,17)], engine = 'python')  
  # Usually, Vertical Ground React Force VGRF values less than 20N are generated from noise
  one_walk.where(one_walk > 20, 0, inplace=True)
  # total right valuees
  one_walk['Total_Force_Right'] = one_walk[list(one_walk.columns[8:16])].sum(axis=1)
  # total left values
  one_walk['Total_Force_Left'] = one_walk[list(one_walk.columns[0:8])].sum(axis=1)

  one_walk_lists=[]*18
  for column in one_walk.columns.values.tolist():
    one_walk_lists.append(one_walk[column].tolist()) 
  right_stance_indices = []
  # right foot phases
  right_phases = [list(g) for k, g in groupby(one_walk_lists[17], lambda x:x>0)]
  right_swings_time=[]*(int(len(right_phases)/2)+1)
  right_stride_time=[]*(int(len(right_phases)/2)+1)
  if (right_phases[0][0] == 0):
    for i in range (0,len(right_phases),2):
      right_swings_time.append((len(right_phases[i])-1)*0.01)
    for i in range (0,len(right_phases)-2,2):
      right_stride_time.append( (len(right_phases[i])+len(right_phases[i+1])-2) *0.01)
  else:
    for i in range (1,len(right_phases),2):
      right_swings_time.append((len(right_phases[i])-1)*0.01)
    for i in range (0,len(right_phases)-2,2):
      right_stride_time.append( (len(right_phases[i])+len(right_phases[i+1])-2) *0.01)
  right_stances = [l for l in right_phases if any(l)]

  # left foot phases
  left_phases = [list(g) for k, g in groupby(one_walk_lists[18], lambda x:x>0)]
  left_swings_time=[]*(int(len(left_phases)/2)+1)
  left_stride_time=[]*(int(len(left_phases)/2)+1)
  if (left_phases[0][0] == 0):
    for i in range (0,len(left_phases),2):
      left_swings_time.append((len(left_phases[i])-1)*0.01)
    for i in range (0,len(left_phases)-2,2):
      left_stride_time.append( (len(left_phases[i])+len(left_phases[i+1])-2) *0.01)
  else:
    for i in range (1,len(left_phases),2):
      left_swings_time.append((len(left_phases[i])-1)*0.01)
    for i in range (0,len(left_phases)-2,2):
      left_stride_time.append( (len(left_phases[i])+len(left_phases[i+1])-2) *0.01)
  left_stances = [l for l in left_phases if any(l)]

  data.append( [id,level,parkinson]
              + one_walk_lists
              + [right_stances]
              + [left_stances]
              + [right_swings_time] + [right_stride_time]
              + [left_swings_time] + [left_stride_time]
              + [ mean (right_swings_time) , np.std(right_swings_time)
              , mean (right_stride_time) , np.std(right_stride_time)
              , mean (left_swings_time) , np.std(left_swings_time)
              , mean (left_stride_time) , np.std(left_stride_time)])

0it [00:00, ?it/s]

In [ ]:
data_df = pd.DataFrame(data, columns =["ID", "level", "y", "L1", "L2","L3", "L4","L5","L6","L7","L8",
                                       "R1", "R2","R3", "R4","R5","R6","R7","R8","Total_Force_Right","Total_Force_Left",
                                       "right_stances"
              , "left_stances"
              , "right_swings_time" , "right_stride_time"
              , "left_swings_time" , "left_stride_time"
              , "mean_right_swings_time" , "std_right_swings_time"
              , "mean_right_stride_time" , "std_right_stride_time"
              , "mean_left_swings_time" , "nstd_left_swings_time"
              , "mean_left_stride_time" , "std_left_stride_time" ])

In [ ]:
data_df.head()

,ID,level,y,L1,L2,L3,L4,L5,L6,L7,...,left_swings_time,left_stride_time,mean_right_swings_time,std_right_swings_time,mean_right_stride_time,std_right_stride_time,mean_left_swings_time,nstd_left_swings_time,mean_left_stride_time,std_left_stride_time
0,GaPt03,3.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[58.08, 46.64, 32.45, 0.0, 0.0, 0.0, 0.0, 0.0,...","[34.1, 23.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[116.82, 116.82, 115.17, 111.87, 105.38, 94.38...","[39.71, 36.52, 33.33, 28.6, 23.76, 0.0, 0.0, 0...","[134.53, 140.25, 145.97, 148.83, 154.55, 157.4...","[188.98, 192.72, 200.2, 203.94, 207.68, 211.42...",...,"[0.5, 0.52, 0.53, 0.48, 0.5, 0.48, 0.53, 0.51,...","[0.73, 1.4000000000000001, 1.46, 1.37, 1.45, 1...",0.406905,0.142662,1.418434,0.411648,0.406905,0.142662,1.418434,0.411648
1,SiPt02,2.5,1,"[108.02, 112.42, 116.38, 120.34, 124.3, 128.26...","[306.79, 315.04, 321.2, 329.45, 338.03, 344.74...","[86.24, 86.24, 84.04, 81.84, 79.53, 77.33, 75....","[48.07, 48.07, 48.07, 48.07, 48.07, 50.16, 50....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[69.19, 69.19, 69.19, 69.19, 69.19, 66.77, 66....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.37, 0.37, 0.35000000000000003, 0.38, 0.3500...","[1.1500000000000001, 1.1300000000000001, 1.09,...",0.322342,0.065292,1.067727,0.183534,0.322342,0.065292,1.067727,0.183534
2,SiPt04,2.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.53, 0.49, 0.45, 0.47000000000000003, 0.45, ...","[1.42, 1.2, 1.1300000000000001, 1.150000000000...",0.457019,0.031831,1.147864,0.065220,0.457019,0.031831,1.147864,0.065220
3,GaPt04,2.5,1,"[0.0, 0.0, 24.75, 41.36, 68.75, 118.36, 166.1,...","[0.0, 0.0, 0.0, 0.0, 0.0, 22.55, 39.49, 54.56,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 27.5, 43.23, 53...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.01, 0.41000000000000003, 0.39, 0.4, 0.42, 0...","[0.79, 1.2, 1.17, 1.18, 1.17, 1.2, 1.21, 1.21,...",0.386237,0.061082,1.280543,0.158604,0.386237,0.061082,1.280543,0.158604
4,SiPt05,2.5,1,"[28.93, 28.93, 28.93, 28.93, 28.93, 28.93, 28....","[60.5, 60.5, 60.5, 60.5, 60.5, 60.5, 60.5, 60....","[79.53, 79.53, 79.53, 79.53, 81.84, 81.84, 81....","[119.9, 119.9, 119.9, 119.9, 119.9, 119.9, 119...","[112.31, 112.31, 112.31, 112.31, 112.31, 112.3...","[55.11, 55.11, 55.11, 55.11, 55.11, 55.11, 55....","[79.86, 79.86, 79.86, 79.86, 79.86, 79.86, 79....",...,"[0.47000000000000003, 0.46, 0.45, 0.45, 0.45, ...","[1.81, 1.3, 1.21, 1.2, 1.21, 1.19, 1.19, 1.17,...",0.378660,0.095352,1.228958,0.889715,0.378660,0.095352,1.228958,0.889715


In [ ]:
len(data_df["right_stances"][0])

84